In [1]:
!source activate tensorflow-gpu

Could not find conda environment: tensorflow-gpu
You can list all discoverable environments with `conda info --envs`.



In [ ]:
!mkdir checkpoint

In [ ]:
glances/htop

In [ ]:
git pull repo
upload some pictures
source activate
mkdir test_dir
mkdir test_dir/checkpoint

In [ ]:
time python style.py --style examples/style/rain_princess.jpg  \
    --checkpoint-dir checkpoint/checkpoints/ --checkpoint-iterations 5 \
    --test examples/style/udnie.jpg  --test-dir checkpoint/ \
    --batch-size 20 --device /gpu:0 --content-weight 1.2e1 \
    &> checkpoint/runtime.txt

In [ ]:
python style.py --style ../styles/wave.jpg \
    --checkpoint-dir test_dir_wave_1_GPU/checkpoint_wave/ --checkpoint-iterations 50 \
    --test ../input_images/stata_modified.jpg --test-dir test_dir_wave_1_GPU/ \
    --batch-size 20 --device /gpu:0 --content-weight 1.2e1 \
    &> test_dir_wave_1_GPU/runtime_wave.txt

In [ ]:
time python style.py --style examples/style/rain_princess.jpg \
    --checkpoint-dir checkpoint/checkpoints/ --checkpoint-iterations 50 --batch-size 20 --device /gpu:0 \
    --test examples/style/udnie.jpg \
    &> checkpoint/runtime.txt

In [ ]:
time python style.py --style ../styles/wave.jpg \
    --checkpoint-dir test_dir_wave_1_GPU/checkpoint_wave/ --checkpoint-iterations 50 \
    --test ../input_images/stata_modified.jpg --test-dir test_dir_wave_1_GPU/ \
    --batch-size 20 --device /gpu:0 --content-weight 1.2e1 \
    &> test_dir_wave_1_GPU/runtime_wave.txt

time python style.py --style ../styles/wave.jpg \
    --checkpoint-dir test_dir_wave_1_CPU/checkpoint_wave/ --checkpoint-iterations 5 \
    --test ../input_images/stata_modified.jpg --test-dir test_dir_wave_1_CPU/ \
    --batch-size 20 --device /cpu:1 --content-weight 1.2e1 \
    &> test_dir_wave_1_CPU/runtime_wave.txt

time python style.py --style ../styles/wave.jpg \
    --checkpoint-dir test_dir_wave_32_CPU/checkpoint_wave/ --checkpoint-iterations 5 \
    --test ../input_images/stata_modified.jpg --test-dir test_dir_wave_32_CPU/ \
    --batch-size 20 --device /cpu:30 --content-weight 1.2e1 \
    &> test_dir_wave_32_CPU/runtime_wave.txt

In [ ]:
picture
test_dir
test_image; test_dir
GPU number
runtime

### Blank Image

In [ ]:
time python style.py --style ../styles/starry_night_by_van_gogh.jpg \
    --checkpoint-dir test_dir_blank/checkpoint_blank/ --checkpoint-iterations 1 \
    --test ../input_images/beach_palm.jpg --test-dir test_dir_blank/ \
    --batch-size 1 --device /cpu:30 \
    &> test_dir_blank/runtime_blank.txt

In [ ]:
time python style.py --style ../styles/wave.jpg \
    --checkpoint-dir test_dir_blank/checkpoint_blank/ --checkpoint-iterations 1 \
    --test ../input_images/stata_modified.jpg --test-dir test_dir_blank/ \
    --batch-size 1 --device /cpu:30 \
    &> test_dir_blank/runtime_blank.txt

In [ ]:
# There is big evaluation time differences between pictures of same shape vs pictures of different shape. 
# Same shape evaluates much faster than different shape.
# It appears that GPU also uses noticeable amounts of CPU.
# CPU currently evalutes using all CPUs, not 1 CPU. Hence, the ratio would be suspectible to number of CPUs.b

do you want to save the models so you can immediately apply style transfer on known style?

In [ ]:
# Plotting

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from collections import OrderedDict

def get_structured_data(data):
    key_value_tups = (OrderedDict(element.split(' = ') for element in line.split('; ')) for line in data)
    return pd.DataFrame(key_value_tups)

with open('test_dir_replicate/runtime_replicate.txt') as f: ## hard coded file name
    data = f.read().splitlines()
    useful_data = [line for line in data if 'Current Time' in line]
    structured_data = get_structured_data(useful_data)
    structured_data[['Epoch', 'Iteration']] = structured_data[['Epoch', 'Iteration']].astype('int')
    structured_data[['Loss', 'Time Elapsed']] = structured_data[['Loss', 'Time Elapsed']].astype('float32')
    structured_data.set_index(structured_data.index * 50, inplace=True) ### hard coded to 50 iterations

In [ ]:
structured_data['Time Elapsed'].cumsum().plot(title="Time vs Iteration", marker='o', linestyle='None')
plt.show()
structured_data['Loss'].plot(logy=True, title='Logged Loss over Iteration')
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from collections import OrderedDict


with open('test_dir_replicate/runtime_replicate.txt') as f: ## hard coded file name
    data = f.read().splitlines()
    useful_lines = [line for line in data if "Loss values" in line]
    key_value_tups = (OrderedDict(element.split(' = ') for element in line.replace("Loss values: ", "").split('; '))
                      for line in useful_lines)
    structured_data_loss = pd.DataFrame(key_value_tups).astype('float')
    structured_data_loss.set_index(structured_data_loss.index * 50, inplace=True) ### hard coded to 50 iterations

structured_data_loss.plot(logy=True)
structured_data_loss.sum(axis=1).plot(logy=True, label='Total loss')
plt.legend()

In [ ]:
structured_data['Loss'][5:].plot()
plt.show()

structured_data_loss['style'][5:].plot()
structured_data_loss['content'][5:].plot()
plt.legend()
plt.show()

In [ ]:
write script to detect new model and rename it to keep it from being overwritten
possible for styel to be applied to video but that's more work

In [ ]:
%%time
import imageio
from datetime import datetime, timedelta

seconds_between_frames = 30 # represents seconds of time for training
max_length_in_seconds = 60 * 60 * 5 ## hard coded to represent maximum length of training time
test_dirname = 'test_dir_replicate/' ### hard coded
output_filename = test_dirname + 'movie.mp4'

image_filenames = []
next_frame_timestamp = None
for row_id, row in structured_data.iterrows():
    if row_id == 0:
        start_time = (datetime.strptime(row['Current Time'], "%Y %B %d, %H:%M:%S") -
                      timedelta(seconds=int(row['Time Elapsed'])))
        next_frame_timestamp = start_time
        while next_frame_timestamp < datetime.strptime(row['Current Time'], "%Y %B %d, %H:%M:%S"):
            image_filenames.append("{}{}_{}.png".format(test_dirname, row['Epoch'], row['Iteration']))
            next_frame_timestamp += timedelta(seconds=seconds_between_frames)
            
    else:
        while next_frame_timestamp < datetime.strptime(row['Current Time'], "%Y %B %d, %H:%M:%S"):
            image_filenames.append("{}{}_{}.png".format(test_dirname, row['Epoch'], row['Iteration']))
            next_frame_timestamp += timedelta(seconds=seconds_between_frames)        
        
image_sequence = []
for filename in image_filenames[:max_length_in_seconds // seconds_between_frames]:
    image_sequence.append(imageio.imread(filename))
imageio.mimsave(output_filename, image_sequence, fps=24) ### fps hard coded

"""
import imageio

sorted_frame_names = 'test_dir_replicate/' + structured_data.apply(
    lambda row: "{}_{}.png".format(row['Epoch'], row['Iteration']), axis=1) # test_dir hard coded
images = []
for filename in sorted_frame_names:
    images.append(imageio.imread(filename))
imageio.mimsave('test_dir_replicate/movie.mp4', images, fps=2) ### fps hard coded
""";

In [ ]:
how about many known styles and then create a grid of transformed images

In [ ]:
ideally a simple web app to upload a picture and watch the style transform

In [ ]:
there's crop video and stitch video code

In [ ]:
import ffmpeg

(ffmpeg
    .input(input_video_path)
    .crop(x1, y1, x_delta, y_delta) # this requires left upper corner and deltas
    .output(output_video_path)
    .run()
)

In [ ]:
import moviepy.editor as mpe
video_width, video_height = mpe.VideoFileClip(input_video_path).size

In [ ]:
%%time
import ffmpeg
import moviepy.editor as mpe
import os



def crop_video(input_video_path, ith_video, jth_horizontal_partition, 
        kth_vertical_partition, num_horizontal_partitions, num_vertical_partitions):    
    video_dir = os.path.dirname(input_video_path)
    video_filename = os.path.basename(input_video_path)
    
    video_width, video_height = mpe.VideoFileClip(input_video_path).size
    y_delta = int(video_height / num_horizontal_partitions)
    x_delta = int(video_width / num_vertical_partitions)
    x1 = kth_vertical_partition * x_delta
    y1 = jth_horizontal_partition * y_delta

    output_video = '{}_{}'.format(ith_video, video_filename)
    output_video_path = os.path.join(video_dir, output_video)
    if os.path.isfile(output_video_path):
        os.remove(output_video_path)
    (ffmpeg
        .input(input_video_path)
        .crop(x1, y1, x_delta, y_delta) # this requires left upper corner and deltas
        .output(output_video_path)
        .run()
    )

if __name__ == "__main__":
    num_horizontal_partitions = 2 ### define here
    num_vertical_partitions = 2 ### define here
    input_video_path = '../input_video/play_dead.mp4' ### define here

    ith_video = 0
    for jth_horizontal_partition in range(num_horizontal_partitions):
        for kth_vertical_partition in range(num_vertical_partitions):
            crop_video(input_video_path, ith_video, 
                       jth_horizontal_partition, kth_vertical_partition, 
                       num_horizontal_partitions, num_vertical_partitions)
            ith_video += 1